In [3]:
def LenOfEpoch(data_sampling_freq, epoch_in_sec):
    return data_sampling_freq*epoch_in_sec

def EDFToCSV(path_in, path_out, len_of_epoch):
    for idx, filename in enumerate(glob.glob(os.path.join(path, '*.edf'))):
        f = pyedflib.EdfReader(filename)

        n = f.signals_in_file
        signal_labels = f.getSignalLabels()
        n_min = f.getNSamples()[0]
        sigbufs = [np.zeros(f.getNSamples()[i]) for i in np.arange(n)]
        for i in np.arange(n):
            sigbufs[i] = f.readSignal(i)
            if n_min < len(sigbufs[i]):
                n_min = len(sigbufs[i])
        f._close()
        del f
#         EOGL = sigbufs[0]
#         EOGR = sigbufs[1]
#         EMR = sigbufs[2]
#         Oz = sigbufs[3]
#         Cz = sigbufs[4]
#         Fpz = sigbufs[5]
#         C3 = sigbufs[6]
#         C4 = sigbufs[7]
#         single_channel = np.subtract(C3, C4)
        
        size = math.ceil(len(single_channel)/len_of_epoch)
        SCEEG_epochs = np.zeros((size, len_of_epoch))
        SCEEG_epochs = [single_channel[i:i + len_of_epoch] for i in range(0, len(single_channel), len_of_epoch)]

        epochs_to_csv = pd.DataFrame(SCEEG_epochs)
        epochs_to_csv = epochs_to_csv.transpose()
        epochs_to_csv.to_csv(path_out+'EEG-'+str(idx+1)+'-epochs.csv')
        print("processing document " +str(idx+1)+' ...')
    print("Completed!")
    
def ExtractTimeFeatures(path_in, len_of_epoch):
    time_features = []
    print('\n Starting feature extraction of .csv files...')
    nr_of_epochs = []
    for idx, filename in enumerate(glob.glob(os.path.join(path, '*.csv'))):
        df = pd.read_csv(filename, index_col=0)
        print('\n processing file '+str(idx+1))
        size = math.ceil((df).size/len_of_epoch)    
    #     print(df.iloc[:, 1297])
        for i in range(size):
            sys.stdout.flush()

            epoch = np.asarray(df.iloc[:, i])
            epoch = epoch[~np.isnan(epoch)]
            mean = np.mean(epoch)
            std = np.std(epoch)
            tw_five = np.percentile(epoch,  25)  
            sv_five = np.percentile(epoch,  75)  
            kurtosis = scp.stats.kurtosis(epoch)
            skewness = scp.stats.skew(epoch)
            minimum = np.amin(epoch)
            maximum = np.amax(epoch)
            ZCR = (np.nonzero(np.diff(epoch > 0))[0]).size
            first_deriv = np.diff(epoch)
            second_deriv = np.diff(epoch,2)

            var_zero = np.mean(epoch ** 2)
            var_d1 = np.mean(first_deriv ** 2)
            var_d2 = np.mean(second_deriv ** 2)

            activity = var_zero
            morbidity = np.sqrt(var_d1 / var_zero)
            complexity = np.sqrt(var_d2 / var_d1) / morbidity        
            time_features.append([mean, std, tw_five, sv_five, kurtosis, skewness, minimum, maximum, ZCR, activity, morbidity, complexity])
            sys.stdout.write("\r%d%% epochs processed" % i)
    print('\n Completed.')
    return time_features

def extractBandFeatures(path_in, len_of_epoch, fs):
    Freq_bands = []
    print('\n Starting feature extraction of .csv files...')
    for idx, filename in enumerate(glob.glob(os.path.join(path, '*.csv'))):
        df = pd.read_csv(filename, index_col=0)
        print('\n processing file '+str(idx+1))
        size = math.ceil((df).size/len_of_epoch)    
        for i in range(size):
            sys.stdout.flush()

            epoch = np.asarray(df.iloc[:, i])
            epoch = epoch[~np.isnan(epoch)]
            fft_vals = np.absolute(np.fft.rfft(epoch)**2)
            ''' If dB is wanted {not reccomended in this case}
            fft_vals_dB = 10*np.log10(fft_vals)
            '''
            fft_freq = np.fft.rfftfreq(int(len(epoch)), 1.0/fs)

            eeg_bands = {'Delta': (1., 4.),
                 'Theta': (4., 8.),
                 'Alpha': (8., 14.),
                 'Beta': (14., 30.),
                 'Gamma': (30., 45.)}
            eeg_band_fft = dict()

            for band in eeg_bands:  
                freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & 
                                   (fft_freq <= eeg_bands[band][1]))[0]
                eeg_band_fft[band] = np.mean(fft_vals[freq_ix])
            Freq_bands.append([eeg_band_fft[band] for band in eeg_bands])            
            
            sys.stdout.write("\r%d%% epochs processed" % i)
    print('\n Completed.')
    return Freq_bands

def extractARFeatures(path_in, len_of_epoch, order_AR):
    AR_coeff = []
    print('\n Starting feature extraction of .csv files...')
    for idx, filename in enumerate(glob.glob(os.path.join(path, '*.csv'))):
        df = pd.read_csv(filename, index_col=0)
        print('\n processing file '+str(idx+1))
        size = math.ceil((df).size/len_of_epoch)    
        for i in range(size):
            sys.stdout.flush()

            epoch = np.asarray(df.iloc[:, i])
            epoch = epoch[~np.isnan(epoch)]
            AR, var, k = aryule(epoch, order_AR)
            AR_coeff.append(AR)      
            sys.stdout.write("\r%d%% epochs processed" % i)
    print('\n Completed.')
    return AR_coeff

def extractSTFTFeatures(fs, window_size):
    STFT = []
    overlap = 0.75*window_size
    f, t, Zxx = signal.stft(epoch, fs, window='hamming', nperseg=window_size, detrend=False, boundary='zeros')
    stft_x = np.abs(Zxx)**2
    stft_flat = stft_x.flatten()
    pos = stft_flat.argsort()[-15:][::-1]
    STFT.append(pos)
    return STFT